## 数据清洗

In [1]:
#-*- coding: utf-8 -*-
# 数据清洗，过滤掉不符合规则的数据

import pandas as pd
'''
------------------------------------------------------------------------------------
路径改一下
------------------------------------------------------------------------------------
'''
datafile = r'C:\Users\DELL\Desktop\air_data.csv'  # 航空原始数据,第一行为属性标签
cleanedfile = r'C:\Users\DELL\Desktop\data_cleaned.csv'  # 数据清洗后保存的文件
cleanedfile2 = r'C:\Users\DELL\Desktop\data_cleaned.xls' 

# 读取原始数据，指定UTF-8编码（需要用文本编辑器将数据装换为UTF-8编码）
data = pd.read_csv(datafile, encoding='utf-8')

# NOTE: * instead of &
data = data[data['SUM_YR_1'].notnull() & data['SUM_YR_2'].notnull()
            ]  # 票价非空值才保留

# 只保留票价非零的，或者平均折扣率与总飞行公里数同时为0的记录。
index1 = data['SUM_YR_1'] != 0
index2 = data['SUM_YR_2'] != 0
index3 = (data['SEG_KM_SUM'] == 0) & (data['avg_discount'] == 0)  # 该规则是“与”
data = data[index1 | index2 | index3]  # 该规则是“或”

# to_csv
data.to_csv(cleanedfile, encoding='utf-8_sig')  # 导出结果
data.to_excel(cleanedfile2)
print('END')

END


In [2]:
import pandas as pd
import numpy as np

# 读取数据清洗后的数据
'''
------------------------------------------------------------------------------------
路径改一下
------------------------------------------------------------------------------------
'''
cleanedfile = r'C:\Users\DELL\Desktop\data_cleaned.csv'  # 数据清洗后保存的文件路径
airline = pd.read_csv(cleanedfile, encoding='utf-8')
# 选取需求属性
airline_selection = airline[['LOAD_TIME','FFP_DATE','LAST_TO_END',
                                     'FLIGHT_COUNT','SEG_KM_SUM','avg_discount']]
print('筛选的属性前5行为：\n',airline_selection.head())

筛选的属性前5行为：
    LOAD_TIME   FFP_DATE  LAST_TO_END  FLIGHT_COUNT  SEG_KM_SUM  avg_discount
0  2014/3/31  2006/11/2            1           210      580717      0.961639
1  2014/3/31  2007/2/19            7           140      293678      1.252314
2  2014/3/31   2007/2/1           11           135      283712      1.254676
3  2014/3/31  2008/8/22           97            23      281336      1.090870
4  2014/3/31  2009/4/10            5           152      309928      0.970658


In [3]:
# 构造属性L
L = pd.to_datetime(airline_selection['LOAD_TIME']) - \
pd.to_datetime(airline_selection['FFP_DATE'])
L = L.astype('str').str.split().str[0]
L = L.astype('int')/30

# 合并属性
airline_features = pd.concat([L,airline_selection.iloc[:,2:]],axis=1)
airline_features['LAST_TO_END'] = airline_features['LAST_TO_END']/30
airline_features.columns = ['L','R','F','M','C']
print('构建的LRFMC属性前5行为：\n',airline_features.head())

# 标准化处理
data = airline_features
data = (data - data.mean(axis = 0))/(data.std(axis = 0)) #简洁的语句实现标准化变换
data.columns=['Z'+i for i in data.columns] # 表头重命名
'''
------------------------------------------------------------------------------------
路径改一下
------------------------------------------------------------------------------------
'''
data.to_excel(r'C:\Users\DELL\Desktop\zscoreddata.xls', index = False)

构建的LRFMC属性前5行为：
            L         R    F       M         C
0  90.200000  0.033333  210  580717  0.961639
1  86.566667  0.233333  140  293678  1.252314
2  87.166667  0.366667  135  283712  1.254676
3  68.233333  3.233333   23  281336  1.090870
4  60.533333  0.166667  152  309928  0.970658


In [4]:
# 画出特征雷达图，代码接KMeans_cluster.py

def print_cluster_result(data, kmodel):
    import pandas as pd
    # 简单打印结果
    r1 = pd.Series(kmodel.labels_).value_counts()  # 统计各个类别的数目
    r2 = pd.DataFrame(kmodel.cluster_centers_)  # 找出聚类中心
    r = pd.concat([r2, r1], axis=1)  # 横向连接（0是纵向），得到聚类中心对应的类别下的数目
    r.columns = list(data.columns) + [u'类别数目']  # 重命名表头
    print(r)
    
    # print(kmodel.cluster_centers_)  # 查看聚类中心
    # print('labels_=', kmodel.labels_)  # 查看各样本对应的类别

def plot_cluster(data, kmodel):
    import numpy as np
    import matplotlib.pyplot as plt    
    plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
    # plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

    labels = data.columns  # 标签
    k = 5  # 数据个数
    plot_data = kmodel.cluster_centers_
    color = ['b', 'g', 'r', 'c', 'y']  # 指定颜色
    
    angles = np.linspace(0, 2 * np.pi, k, endpoint=False)
    plot_data = np.concatenate((plot_data, plot_data[:, [0]]), axis=1)  # 闭合
    angles = np.concatenate((angles, [angles[0]]))  # 闭合
    
    fig = plt.figure()
    ax = fig.add_subplot(111, polar=True)  # polar参数！！
    for i in range(len(plot_data)):
        ax.plot(angles, plot_data[i], 'o-', color=color[i],
                label=u'客户群' + str(i+1), linewidth=2)  # 画线
    
    ax.set_rgrids(np.arange(0.01, 3.5, 0.5),
                  np.arange(-1, 2.5, 0.5), fontproperties="SimHei")
    ax.set_thetagrids(angles * 180 / np.pi, labels, fontproperties="SimHei")
    plt.legend(loc=4)
    
    
    '''
    ____________________________________________________________________________________
    加了保存图片代码，这里路径改一下就行，不需要就注释掉
    ————————————————————————————————————————————————————————————————————————————————————
    '''
    fig = plt.gcf()
    fig.savefig(r'C:\Users\DELL\Desktop\radar.png', figsize=(15,15),dpi=200)
    
    plt.show()

In [5]:
# -*- coding: utf-8 -*-
# K-Means聚类算法

import pandas as pd
from sklearn.cluster import KMeans  # 导入K均值聚类算法
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
'''
------------------------------------------------------------------------------------
路径改一下
------------------------------------------------------------------------------------
'''
inputfile = r'C:\Users\DELL\Desktop\zscoreddata.xls'  # 待聚类的数据文件
k = 5  # 需要进行的聚类类别数，需结合业务的理解和分析来确定客户的类别数量

# 读取数据并进行聚类分析
data = pd.read_excel(inputfile)  # 读取数据

# 调用k-means算法，进行聚类分析
# TODO: error here
# n_jobs是并行数，一般等于CPU数较好 , window下多进程 跑失败
kmodel = KMeans(n_clusters=k, n_jobs=1)  
kmodel.fit(data)  # 训练模型

print_cluster_result(data, kmodel)
plot_cluster(data, kmodel)
print('END')

         ZL        ZR        ZF        ZM        ZC   类别数目
0 -0.312872  1.687311 -0.573893 -0.536735 -0.176233  12105
1  1.161296 -0.377455 -0.086538 -0.094390 -0.157523  15726
2 -0.700439 -0.415281 -0.160508 -0.160194 -0.258967  24594
3  0.483475 -0.799422  2.482936  2.424166  0.310327   5338
4  0.039279 -0.001717 -0.233755 -0.238419  2.161024   4281


<Figure size 640x480 with 1 Axes>

END
